# Improvement Trial With Transfer of Successful AI

This trial will test transferring AI from more successful AI to less successful ones.

In [1]:
import gpu_rec
from constants import *
from copy import deepcopy
from random import shuffle
from game import Game
import numpy as np
import sys

Using TensorFlow backend.


In [2]:
#game is now using lag=1 instead of lag=2 for q-score
#maybe this will help guide it to higher point values in short term, 
#with less opportunities for probabilities straying
test_config = deepcopy(DEFAULT_GAME_CONFIG)
test_config['memory'] = 1
test_config['discount'] = 0.01

init_game = Game(n_players=5, q_lag=1, config=test_config)
#goal: eventually train players to not want to grab more tickets

players = init_game.players
#former will be shuffled
player_list = [p for p in players]
overall_player_wins = []
overall_player_scores = []
overall_winning_scores = []
overall_game_turns = []



In [3]:
players[0].ai.receive_models_from_other_ai(players[1].ai)

In [4]:
def which_min(x):
    min_val = min(x)
    return x.index(min_val)

def which_max(x):
    max_val = max(x)
    return x.index(max_val)

def aid_weakest_ai(players, wins):
    players[which_min(wins)].ai.receive_models_from_other_ai(players[which_max(wins)].ai)
    

In [5]:
temperatures = [ 0.2, 0.1, 0.1, 0.05, 0.01, 0.01, 0.01, 0.01, 0.001, 0.001, 0.001, 0.001, 0., 0., 0.]
n_trials_per_cycle = 80
n_cycles = 3

for temp_index, temperature in enumerate(temperatures):
    print '---TEMPERATURE: %.03f---' % temperature
    print '+IDX=%d+' % temp_index
    if temp_index==9:
        test_config['discount'] = 0.05
    elif temp_index==10:
        test_config['discount'] = 0.1
    elif temp_index==11:
        test_config['discount'] = 0.2
    elif temp_index==12:
        test_config['discount'] = 0.5
    elif temp_index==13:
        test_config['discount'] = 1
    test_config['temperature'] = temperature
    player_wins = [0 for _ in players]
    winning_scores = []
    player_scores = [[] for _ in players]
    game_turns = []
    for cycle in range(n_cycles):
        print 'in cycle %d' % cycle
        for i in range(n_trials_per_cycle):
            sys.stdout.write('.')
            shuffle(players)
            game = Game(pre_existing_players=players, config=test_config, separated_models=True)
            game.run(False)
            for i, player in enumerate(player_list):
                player_wins[i]+= player.win
                player_scores[i].append(player.total_points)
            winning_scores.append(game.winning_score)
            game_turns.append(game.turn)
        #don't train player 0, train player 1 partially, and train players 2 and 3 much more
        for i, player in enumerate(player_list):
            if i==1:
                player.ai.train_win()
                player.ai.train_q()
            elif i > 1:
                player.ai.train_win(20)
                player.ai.train_q(20)
    for player in player_list:
        player.ai.reset_history()
    aid_weakest_ai(player_list[1:], player_wins[1:])
    overall_player_wins.append(player_wins)
    overall_player_scores.append(player_scores)
    overall_winning_scores.append(winning_scores)
    overall_game_turns.append(game_turns)
    #print out some summaries
    print 'WINNING SCORE AVERAGE: %.1f' % np.mean(winning_scores)
    print 'NUMBER OF TURNS AVERAGE: %.1f' % np.mean(game_turns)
    for i, player in enumerate(player_list):
        print '---PLAYER %d STATS---' % i
        print 'WIN AVERAGE: %.3f' % (np.mean(player_wins[i])/(float(n_trials_per_cycle*n_cycles)))
        print 'SCORE AVERAGE: %.1f' % np.mean(player_scores[i])
        

---TEMPERATURE: 0.200---
+IDX=0+
in cycle 0
................................................................................in cycle 1
................................................................................in cycle 2
................................................................................WINNING SCORE AVERAGE: 4.0
NUMBER OF TURNS AVERAGE: 261.8
---PLAYER 0 STATS---
WIN AVERAGE: 0.371
SCORE AVERAGE: -13.7
---PLAYER 1 STATS---
WIN AVERAGE: 0.396
SCORE AVERAGE: -14.1
---PLAYER 2 STATS---
WIN AVERAGE: 0.183
SCORE AVERAGE: -22.8
---PLAYER 3 STATS---
WIN AVERAGE: 0.062
SCORE AVERAGE: -40.4
---PLAYER 4 STATS---
WIN AVERAGE: 0.008
SCORE AVERAGE: -42.4
---TEMPERATURE: 0.100---
+IDX=1+
in cycle 0
................................................................................in cycle 1
................................................................................in cycle 2
................................................................................WINNING SCORE AVERAGE: 7.

ai.py:414: RuntimeWarning: invalid value encountered in divide
  return (selected)/np.sum(selected)
ai.py:386: RuntimeWarning: invalid value encountered in less
  return np.random.choice(len(new_probs), p=new_probs)


...............................................................................in cycle 1
................................................................................in cycle 2
................................................................................WINNING SCORE AVERAGE: 16.3
NUMBER OF TURNS AVERAGE: 223.5
---PLAYER 0 STATS---
WIN AVERAGE: 0.054
SCORE AVERAGE: -31.9
---PLAYER 1 STATS---
WIN AVERAGE: 0.217
SCORE AVERAGE: -11.8
---PLAYER 2 STATS---
WIN AVERAGE: 0.196
SCORE AVERAGE: -12.0
---PLAYER 3 STATS---
WIN AVERAGE: 0.267
SCORE AVERAGE: -9.2
---PLAYER 4 STATS---
WIN AVERAGE: 0.279
SCORE AVERAGE: -8.3
---TEMPERATURE: 0.010---
+IDX=5+
in cycle 0
................................................................................in cycle 1
................................................................................in cycle 2
................................................................................WINNING SCORE AVERAGE: 18.8
NUMBER OF TURNS AVERAGE: 223.9
---PLAYER 0 

still, temperature=0 has problems (though slightly less noticeable after only one iteration of it)

lower temperatures do produce better results, and overall the players perform much better later on.

there may be an issue with not enough training data. I am going to increase the per-cycle sample size next time to accomodate for this

In [6]:
print game.turn
print i
print cycle
print temp_index
for player in players:
    print player
    print len(player.determine_possible_tracks())

230
4
2
14
player with 0/5 completed/uncompleted tickets, 15 train cards, 2 cars, and 
		8 total points
29
player with 0/7 completed/uncompleted tickets, 27 train cards, 8 cars, and 
		3 total points
67
player with 0/7 completed/uncompleted tickets, 17 train cards, 4 cars, and 
		-13 total points
40
player with 0/7 completed/uncompleted tickets, 17 train cards, 5 cars, and 
		-13 total points
52
player with 0/7 completed/uncompleted tickets, 22 train cards, 1 cars, and 
		23 total points
5


In [7]:
np.mean(overall_winning_scores,1)

array([  4.04166667,   7.15833333,   7.04583333,  11.12083333,
        16.27916667,  18.8375    ,  17.325     ,  14.5625    ,
        16.85833333,  16.575     ,  21.49166667,  23.3       ,
        21.38333333,  19.55833333,  19.94583333])

In [8]:
np.mean(overall_player_scores, ( 2,))

array([[-13.69583333, -14.05416667, -22.75      , -40.425     , -42.4       ],
       [-12.7875    , -17.39583333, -26.62916667, -34.51666667,
        -16.64166667],
       [-14.47083333, -16.5375    , -23.68333333, -16.6       , -16.25      ],
       [-14.975     , -17.45833333, -14.74583333, -12.87916667,
        -15.82916667],
       [-31.95      , -11.82083333, -11.99583333,  -9.19583333,
         -8.27083333],
       [-29.55833333, -11.70833333,  -8.80833333, -10.625     ,
         -6.99583333],
       [-31.05      ,  -8.47083333,  -8.09166667, -14.39166667,
         -7.15833333],
       [-28.33333333,  -8.82083333, -11.92083333, -10.04166667,
        -11.87083333],
       [ -9.09166667, -12.22916667, -10.92083333,  -8.90833333, -10.925     ],
       [-10.35833333, -10.125     ,  -9.53333333, -12.3375    ,
        -10.62083333],
       [-23.09583333,  -8.275     ,  -7.8375    ,  -5.9875    ,  -6.4       ],
       [-23.55416667,  -7.97916667,  -7.55      ,  -7.15416667,  -6.2125   

In [9]:
for i, player in enumerate(player_list):
    player.ai.save_models('ai_h5/p%d_test08_%%s.h5' % i)